# Salary investigation - Feature engineering and Machine learning

**Still working. For data analysis and visualisation, please check the "detailed_analysis_visual_on_income" notebook**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
# sklearn.set_config(print_changed_only=True)
%matplotlib inline

In [5]:
df = pd.read_csv("data/adult.csv", index_col=0)
df['income'] = df['income'].str.replace(" ", "")
df.head()

df_process = df.copy()
# Think for this data, because the target column (income) is a binary value, using x-y plot is difficult to 
# see the relationship between x and y. I think it is better to use histogram, with y as colors

In [2]:
# Label encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def label_encode(df, X_col, label_encode_col):
# y_encode_col = 'income'

    LEncoder = LabelEncoder()
    OHEncoder = OneHotEncoder()

    X_label_enc = df[X_col]

    for col in label_encode_col:
        # The column names have a space in front. Clear it
    #     cleaned_col = col.replace(' ', '')
    #     print(cleaned_col)

        ## TODO Fixed the empty space in front of the column name
        temp_dummy = pd.get_dummies(X_label_enc[col],prefix=col,prefix_sep='-')
        cleaned_val = [c.replace(' ','') for c in temp_dummy.columns]
        cleaned_cols = {k:v for (k,v) in zip(temp_dummy.columns, cleaned_val)}
    #     print(cleaned_cols)
        temp_dummy.rename(columns=cleaned_cols, inplace=True)
    #     print('temp_dummy.columns:', temp_dummy.head(2))
        X_label_enc.drop(columns=col, inplace=True)
        X_label_enc = X_label_enc.merge(temp_dummy, left_index=True, right_index=True)
    #     print(X_label_enc.columns)

    #     print(X)
    # Use for loop to fit and trasnform each column that needs to be encoded
    #     X_label_enc[col] = LEncoder.fit_transform(X_label_enc[col])
    #     X_label_enc[col] = OHEncoder.fit_transform(X_label_enc[col].reshape(-1,1))


    # print(pd.get_dummies(X_label_enc['workclass']))

    # LEncoder.fit_transform(df[columns[0]])
    y_encode = LEncoder.fit_transform(y)

    print(X_label_enc.columns)
    
    return X_label_enc, y_encode

In [ ]:
# Not including education because it has strong correlation with education-num
label_encode_col = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'gender']

X_label_enc, y_encode = label_encode(df, X_col, label_encode_col)

print(y_encode)

X_label_enc.head()

In [ ]:
# Native country to be engineered further.
# Maybe use some factors to categorise countries such as: 
# 1. GPP (Gross average per person)
# https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita
# 2. Cultural similarities (maybe simply English-speaking and non-English speaking?)
# https://www.economist.com/graphic-detail/2019/12/04/where-are-the-worlds-best-english-speakers
# https://ceoworld.biz/2019/11/05/revealed-the-worlds-best-non-native-english-speaking-countries-2019/
# 3. Geographical location (continent)


# Easy version
US_check = ['united-states', 'outlying-us(guam-usvi-etc)']

# Need to do strip because there is weird space in entries
temp_native_country = df_process['native-country'].apply(lambda x: 1 if x.lower().strip() in US_check else 0)

native_country_dummy = pd.get_dummies(temp_native_country, prefix='native-country: US or not', prefix_sep='-')


df_process = df_process.merge(native_country_dummy, left_index=True, right_index=True)

# print(df_process['native-country'][0].lower().strip())

# print(df_process['native-country'].value_counts())
# print(temp_n_country)
# print(native_country_dummy)
df_process.head()

In [ ]:
# More complicated method
# Get the GDP and English fluency information
dict_country_GDP = {}
dict_country_Eng_fluency = {}

temp_country_GDP # bin this using qcut
temp_Eng_fluency # bin this using qcut

## Functions that perform vertical quartile quantification and then horizontal binning
Since several columns require the same procedure (age, capital-gain and capital-loss), will write functions here
for calling <br>
For capital-gain and capital-loss, because there are too many different horizontal values (different amount of capital-gain/capital-loss, thus, would just use a horizontal qcut and omit the vertical one. <br><br>
Because most people have 0 capital-gain, most quartiles are concentrated close to 0 too.
By setting duplicate drop, don't get to see a lot of different bins even though quartile cut is big

## Capital-gain and capital-loss analysis
There are too many different values for capital-gain and capital-loss (and quite a few outliers).
So similar to age feature engineering, I will use a vertical quartile quantification method to limit the horizontal ranges.
The amount of money is again a continuous value, so might have to consider to similar things as age featuring
where I do the horizontal binning based on levels

In [6]:
def bin_num_col(df, col, q_num, drop_zero=False):
    # binning numerical columns with qcut. col should just be one column string. q_num is the number of bins to be made
    # if drop_zero=True, set zero as one of the bin then drop it, then perform qcut with the rest
    # print(df_process['capital-gain'])
    # df_process['capital-gain'].value_counts() # Most people have 0 capital-gain (perhaps no investment?)
    binned = []
#     temp_df = df.copy()
    
    if drop_zero == True:
        binned.extend([-0.01, 0.01])
        temp_df = df[(df[col] < -0.01) | (df[col] > 0.01)][col]
        print(temp_df)
    else:
        temp_df = df[col]
    
    if col in df.columns:
        ser, temp_binned = pd.qcut(temp_df, q=q_num, duplicates="drop", retbins=True)
        # With retbins=True, pd.qcut returns a tuple whose second element is the bins
        # The first return element is a series while the second one is the break point
        binned.extend(temp_binned)
        print(binned)

        # bin_cap_gain -= 0.01
        # Need to do this because the cut is excluding the left hand side boundary value, which means 0 would not
        # be include in (0 x]

        binned_col = pd.cut(df[col], bins=binned, include_lowest=True)

        binned_dummy = pd.get_dummies(binned_col,prefix= col + '_binned',prefix_sep='-')
#         df.drop(columns=[col], inplace=True)  
        df = df.merge(binned_dummy, left_index=True, right_index=True)
    
    return df

In [7]:
df_process = bin_num_col(df_process, 'capital-gain', 5, drop_zero=True)
df_process = bin_num_col(df_process, 'capital-loss', 5, drop_zero=True)
df_process = bin_num_col(df_process, 'education-num', 5)

df_process.head()

0         2174
8        14084
9         5178
59        5013
60        2407
84       14344
101      15024
105       7688
106      34095
113       4064
132       4386
197      14084
198       7298
206       7298
208      15024
227       1409
230       3674
263       4064
271       1055
272       2407
276       7298
279       7298
281       5178
285      15024
297       3464
300       7688
303       4386
311       7298
329       2050
341       7298
         ...  
32098     7298
32123    13550
32139    15024
32155     1055
32156     7298
32164    10520
32169     4101
32179     5178
32198    20051
32203    15024
32228     7688
32238    99999
32249     7688
32308     3103
32341    15831
32354     6849
32365    15024
32367     2653
32370    27828
32388     1471
32399     7298
32434     7443
32462     7298
32466    15024
32473     1506
32515     3471
32518    99999
32538    15020
32548     1086
32560    15024
Name: capital-gain, Length: 2712, dtype: int64
[-0.01, 0.01, 114.0, 3103.0, 5013.0, 7

,age,workclass,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,...,"capital-loss_binned-(0.01, 155.0]","capital-loss_binned-(155.0, 1617.0]","capital-loss_binned-(1617.0, 1876.0]","capital-loss_binned-(1876.0, 1902.0]","capital-loss_binned-(1902.0, 2001.0]","capital-loss_binned-(2001.0, 4356.0]","education-num_binned-(0.999, 9.0]","education-num_binned-(9.0, 10.0]","education-num_binned-(10.0, 13.0]","education-num_binned-(13.0, 16.0]"
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,...,0,0,0,0,0,0,0,0,1,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,...,0,0,0,0,0,0,0,0,1,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,...,0,0,0,0,0,0,1,0,0,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,...,0,0,0,0,0,0,1,0,0,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
# Similarly perform the same with capital-loss
df_process['capital-loss'].describe()

bin_cap_loss = pd.qcut(df_process['capital-loss'], q=50, duplicates="drop")

print(bin_cap_loss.value_counts())
print(df_process[df_process['capital-loss'] < 1590]['capital-loss'].value_counts())

# df_process['capital-gain'].value_counts()

### Age feature engineering
The ratio between people earning more than 50K and less than 50K changes with the age group.
With that, binning would be more effective if we can set the boundary at which the ratios change a lot.

1. In this example, all bins have more people earning less than 50K than people earning more than 50K. So the ratio would just be (Number of people earning less than 50K)/(Number of people earning more than 50K).
2. Some of the ratios can be huge (at young age) compared to low ratios. To reduce the effect of big ratio and increase the effect of smaller ratio change, log transfomration would be applied
3. Final step is to apply "Bayesian blocks", which is an adaptive method to find the optimal binning strategy

For a set of histogram bins or blocks, each of an arbirary size, one can use a Bayesain likelihood framework to compute a fitness function which only depeds on two numbers: the width of each block, and the number of points in each block. The edges betwen these blocks (the change-points) can be varied, and the overall block configuration with the maximum fitness is quantitatively the est binning.


In [ ]:
def bayesian_blocks(t):
    """Bayesian Blocks Implementation

    By Jake Vanderplas.  License: BSD
    Based on algorithm outlined in http://adsabs.harvard.edu/abs/2012arXiv1207.5578S

    Parameters
    ----------
    t : ndarray, length N
        data to be histogrammed

    Returns
    -------
    bins : ndarray
        array containing the (N+1) bin edges

    Notes
    -----
    This is an incomplete implementation: it may fail for some
    datasets.  Alternate fitness functions and prior forms can
    be found in the paper listed above.
    """
    # copy and sort the array
    t = np.sort(t)
    N = t.size

    # create length-(N + 1) array of cell edges
    edges = np.concatenate([t[:1],
                            0.5 * (t[1:] + t[:-1]),
                            t[-1:]])
    block_length = t[-1] - edges

    # arrays needed for the iteration
    nn_vec = np.ones(N)
    best = np.zeros(N, dtype=float)
    last = np.zeros(N, dtype=int)

    #-----------------------------------------------------------------
    # Start with first data cell; add one cell at each iteration
    #-----------------------------------------------------------------
    for K in range(N):
        # Compute the width and count of the final bin for all possible
        # locations of the K^th changepoint
        width = block_length[:K + 1] - block_length[K + 1]
        count_vec = np.cumsum(nn_vec[:K + 1][::-1])[::-1]

        # evaluate fitness function for these possibilities
        fit_vec = count_vec * (np.log(count_vec) - np.log(width))
        fit_vec -= 4  # 4 comes from the prior on the number of changepoints
        print(fit_vec)
        fit_vec[1:] += best[:K]

        # find the max of the fitness: this is the K^th changepoint
        i_max = np.argmax(fit_vec)
        last[K] = i_max
        best[K] = fit_vec[i_max]

    #-----------------------------------------------------------------
    # Recover changepoints by iteratively peeling off the last block
    #-----------------------------------------------------------------
    change_points =  np.zeros(N, dtype=int)
    i_cp = N
    ind = N
    while True:
        i_cp -= 1
        change_points[i_cp] = ind
        if ind == 0:
            break
        ind = last[ind - 1]
    change_points = change_points[i_cp:]

    return edges[change_points]

In [ ]:
log_age_ratio_diff = log_age_ratio.to_frame().T.diff(axis=1)

log_age_ratio.plot.bar()
log_age_ratio_diff.T.plot.bar()

In [ ]:
def get_hori_bound(s):
# This function takes in either series or a list of time-series like values that have been quantified with vertical qcut
# To reduce the number of discrete values in binning, a simple boundary values would be returned for horizontal qcut
# Within each boundary, the vertical (y-axis) values would be the same
# At the moment there is no smoothing method, bumpy vertical values within a long stretch of vertical values
# would have its own boundary

# The algorithm is simple too, whenever vertical value is the same, no boundary is set and move on to the next sample
# If the vertical value is different, then a boundary is set and the new vertical value is set
    bin_boundary = []
    prev_yval = None
    labels = []
    
#     curr_
    
    for row in s.iteritems():
        if not prev_yval or not row[1] == prev_yval:
            prev_yval = row[1]
#             print(row.index)
            bin_boundary.append(row[0])   
    

    return bin_boundary

In [ ]:
vert_qcut_val = pd.qcut(log_age_ratio, 5)
print(vert_qcut_val.unique()[0].left)

print(vert_qcut_val.unique())

interval=[]
mid_val = []
labels = []

for v in vert_qcut_val.unique():
    interval.append(v.left)
    interval.append(v.right)
    mid_val.append((v.left + v.right)/2)
    
    
vert_qcut_list = sorted(set(interval))
mid_val_list = sorted(set(mid_val))

print(mid_val_list)
print(labels)

bin_log_age_ratio = pd.cut(x=log_age_ratio, bins=vert_qcut_list, labels=mid_val_list)

# bin_log_age_ratio.to_frame().T.plot.bar()
# test.to_frame().plot.bar()
bin_boundary = get_hori_bound(bin_log_age_ratio)

df_process['age-binned'] = pd.cut(df_process['age'], bins=bin_boundary)

age_binned_dummy = pd.get_dummies(df_process['age-binned'],prefix='age-binned',prefix_sep='-')

# df_process.drop(columns=col, inplace=True)
df_process = df_process.merge(age_binned_dummy, left_index=True, right_index=True)



print(bin_boundary)
# df_process[['age','age_binned']].head()
df_process.head()

In [ ]:
# Perform automatic binning on continuous values

# test = pd.qcut(df['age'].values, 5, \
#                labels=['young', 'prime-time', 'middle-age','retiring', 'senior']) # Categorical if given is array

test = pd.qcut(df['age'], 10) # Categorical if given is array

print(test.value_counts())